In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import itertools
import xlearn as xl
from subprocess import check_output
import gc
# 建議使用Linux或是在Docker內跑這個套件

In [2]:
print(check_output(["ls", "./"]).decode("utf8"))

pyfm.ipynb
readme.txt
test.txt
train.txt
xLearn.ipynb



In [3]:
df = pd.read_csv('train.txt',header=None,delimiter='\t',nrows=100000)

In [4]:
column = ['Label'] + ['I'+str(i+1) for i in range(13)] + ['C'+str(i+1) for i in range(26)]

In [5]:
df.columns = column

In [6]:
df.select_dtypes('object').head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [7]:
def FillNa(df):
    df=df.copy()
    Num_Features=df.select_dtypes(['float','int']).columns.tolist()
    Str_Features=df.select_dtypes(['object']).columns.tolist()
    df[Num_Features]=df[Num_Features].fillna(0)
    for col in Str_Features:
        df[col]=df[col].fillna('NAN')
        df[col]=df[col].replace('XNA','NAN')
    return df

In [8]:
df = FillNa(df)

In [9]:
df.head()

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NAN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NAN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NAN,NAN,e587c466,ad3062eb,3a171ecb,3b183c5c,NAN,NAN
3,0,0.0,893,0.0,0.0,4392.0,0.0,0.0,0.0,0.0,...,1e88c74f,74ef3502,NAN,NAN,6b3a5ca6,NAN,3a171ecb,9117a34a,NAN,NAN
4,0,3.0,-1,0.0,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NAN,NAN,21c9516a,NAN,32c7478e,b34f3128,NAN,NAN


In [10]:
Categories = df.select_dtypes('object').columns.tolist()

In [11]:
Numerics = [i for i in df.columns.tolist() if i not in Categories+['Label']]

In [12]:
Features = [i for i in df.columns.tolist() if i != 'Label']

In [13]:
current_code = len(Numerics)
catdict = {}
catcodes = {}

In [14]:
for x in Numerics:
    catdict[x] = 0
for x in Categories:
    catdict[x] = 1

In [15]:
nrows = df.shape[0]
columns = len(Features)

In [16]:
df.head()

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NAN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NAN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NAN,NAN,e587c466,ad3062eb,3a171ecb,3b183c5c,NAN,NAN
3,0,0.0,893,0.0,0.0,4392.0,0.0,0.0,0.0,0.0,...,1e88c74f,74ef3502,NAN,NAN,6b3a5ca6,NAN,3a171ecb,9117a34a,NAN,NAN
4,0,3.0,-1,0.0,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NAN,NAN,21c9516a,NAN,32c7478e,b34f3128,NAN,NAN


In [ ]:
with open("test_ffm.txt",'w') as text_file:
    for n, r in enumerate(range(nrows)):
        datastring = ""
        datarow = df.loc[r].to_dict()
        datastring += str(int(datarow['Label']))
        for i, x in enumerate(catdict.keys()):
            if(catdict[x]==0):
                datastring = datastring + " " + str(i) + ":" + str(i) + ":" + str(datarow[x])
            else:
                if(x not in catcodes):
                    catcodes[x] = {}
                    current_code += 1
                    catcodes[x][datarow[x]] = current_code
                
                elif(datarow[x] not in catcodes[x]):
                    current_code += 1
                    catcodes[x][datarow[x]] = current_code
                # Let All Data as input form
                code = catcodes[x][datarow[x]]  
                datastring = datastring + " " + str(i) + ":" + str(int(code)) + ":1" 
            
        datastring = datastring + '\n'
        text_file.write(datastring)

In [ ]:
# Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain("./test_ffm.txt")  # Training data

# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
param = {'task':'binary', 'lr':0.2,
         'lambda':0.002, 'metric':'acc'}

# Start to train
# The trained model will be stored in model.out
ffm_model.fit(param, './model.out')

# Prediction task
ffm_model.setSigmoid()  # Convert output to 0-1




In [ ]:
ffm_model

In [ ]:
y = df['Label'].values
del df['Label']
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.2, random_state=7, stratify = y)
gc.collect()

In [ ]:
fm_model = xl.create_fm()

In [ ]:
X = df.loc[:,'C1':].to_dict('records')
v = DictVectorizer()
X = v.fit_transform(X)

In [ ]:
df.head()

In [ ]:
X_train = v.transform(X_train.loc[:,'C1':].to_dict('records'))
X_valid = v.transform(X_valid.loc[:,'C1':].to_dict('records'))

In [ ]:
param = {'task':'binary', 'lr':0.2,
         'lambda':0.002, 'metric':'acc'}

In [ ]:
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain("./small_train.txt")  # Training data
ffm_model.setValidate("./small_test.txt")  # Validation data

In [ ]:
ffm_model.fit(param, './model.out')

In [ ]:
# Evaluate the trained model
test_precision = precision_at_k(model, X_valid, k=5).mean()